<a href="https://colab.research.google.com/github/ansharma72/covid19_analysis/blob/master/Dask_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 81kB/s 
     |████████████████████████████████| 204kB 70.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=7e0ad49f32075fd483ec0685c9886fe7b0a2e95b6ce7bd601c276bebfb37b348
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualization
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)

from matplotlib import rcParams
sns.set(context='notebook', style='whitegrid', rc={'figure.figsize': (18,4)})
rcParams['figure.figsize'] = 18,4

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# setting random seed for notebook reproducability
rnd_seed=23
np.random.seed=rnd_seed
np.random.set_state=rnd_seed

In [ ]:
spark = SparkSession.builder.master("local[2]").appName("data-wrangling").getOrCreate()
spark

In [26]:
!pip install miceforest
!pip install sparkpickle

  Created wheel for sparkpickle: filename=sparkpickle-1.0.1-cp37-none-any.whl size=27391 sha256=27b24356922260528ad97c7511cffefae60fb7c7ca9756454f20e9aa989f3df0
  Stored in directory: /root/.cache/pip/wheels/81/83/f7/189f6eda49941039142c83e56e711a956068009daaff49acb0
Successfully built sparkpickle


In [ ]:
import pandas as pd
import numpy as np
# import tensorflow as tf
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import io
# %cd /content/drive/MyDrive/Ryan - multiinput GAN regression
# import utils
import config
from config import catgoricalValues
from config import NumericColumns
from config import vector

from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [20]:
FullMerge = pd.read_pickle("./FullMerge.pkl.bz2")
emissions = pd.read_pickle("./emissions.pkl.bz2")
FundMapping = pd.read_pickle("./FundMapping.pkl.bz2")


NameError: ignored

In [38]:
FullMerge.to_parquet("FullMerge.parquet")
emissions.to_parquet("Emissions.parquet")
FundMapping.to_parquet("FundMapping.parquet")



In [49]:
FullMergeDf = spark.read.parquet("FullMerge.parquet")
emissionsDf = spark.read.parquet("Emissions.parquet")
FundMappingDf = spark.read.parquet("FundMapping.parquet")

In [50]:

# Parquet files can also be used to create a temporary view and then used in SQL statements.
FullMergeDf.createOrReplaceTempView("FullMerge")
emissionsDf.createOrReplaceTempView("Emissions")
FundMappingDf.createOrReplaceTempView("FundMapping")

# Cache here the 3 data frames 
FullMergeDf.cache
emissionsDf.cache
FundMappingDf.cache

In [65]:
SQL = """ 
          Select * from FullMerge f """
results_fullmerge = spark.sql(SQL)

results_fullmerge.printSchema()
results_fullmerge.count(). # 165,909
results_fullmerge.show(10)

root
 |-- index: long (nullable = true)
 |-- date_x: string (nullable = true)
 |-- filing_date_x: string (nullable = true)
 |-- currency_symbol_x: string (nullable = true)
 |-- totalAssets: string (nullable = true)
 |-- intangibleAssets: string (nullable = true)
 |-- earningAssets: string (nullable = true)
 |-- otherCurrentAssets: string (nullable = true)
 |-- totalLiab: string (nullable = true)
 |-- totalStockholderEquity: string (nullable = true)
 |-- deferredLongTermLiab: string (nullable = true)
 |-- otherCurrentLiab: string (nullable = true)
 |-- commonStock: string (nullable = true)
 |-- retainedEarnings: string (nullable = true)
 |-- otherLiab: string (nullable = true)
 |-- goodWill: string (nullable = true)
 |-- otherAssets: string (nullable = true)
 |-- cash: string (nullable = true)
 |-- totalCurrentLiabilities: string (nullable = true)
 |-- shortTermDebt: string (nullable = true)
 |-- shortLongTermDebt: string (nullable = true)
 |-- shortLongTermDebtTotal: string (nullable =

165909

+-----+----------+-------------+-----------------+--------------+----------------+-------------+------------------+-------------+----------------------+--------------------+----------------+-----------+----------------+-------------+-------------+------------+-------------+-----------------------+-------------+-----------------+----------------------+----------------------+----------------------+------------------+-------------------+-----------------+--------------------+--------------+-------------+-------------+---------------+--------------------+------------------------------------------+------------------------------------------------+-----------------------------------+-----------------------+----------------------+-------------------------+---------------------------+---------------+-----------------------+----------------------+----------------------------+---------------------+-----------------------+-----------------+--------------------------+--------------------------+----

In [68]:
SQL = """ 
          Select * from Emissions e"""
results_emission = spark.sql(SQL)
results_emission.count() # 1611
results_emission.show(10)
results_emission.printSchema()


1611

+-----+----+------------+-------------------+
|index|year|carbonOutput|     reference_name|
+-----+----+------------+-------------------+
|   78|2020|     151.157|       3i Group plc|
|  151|2019|       142.2|       3i Group plc|
|  231|2018|       156.0|       3i Group plc|
|  518|2020|   4050000.0|         3M Company|
|  624|2019|   4790000.0|         3M Company|
|  726|2018|   3960000.0|         3M Company|
| 1516|2020|    533000.0|Abbott Laboratories|
| 1602|2019|    525000.0|Abbott Laboratories|
| 1687|2018|    531000.0|Abbott Laboratories|
| 1775|2020|    314421.0|         AbbVie Inc|
+-----+----+------------+-------------------+
only showing top 10 rows

root
 |-- index: long (nullable = true)
 |-- year: long (nullable = true)
 |-- carbonOutput: double (nullable = true)
 |-- reference_name: string (nullable = true)



In [75]:
SQL = """ 
          Select * from FullMerge f left join Emissions e
          on f.name=e.reference_name and f.year= e.year 
        """
results_full_merge_emission = spark.sql(SQL)
results_full_merge_emission.count()
results_full_merge_emission.show(10)


165909

+-----+----------+-------------+-----------------+--------------+----------------+-------------+------------------+-------------+----------------------+--------------------+----------------+-----------+----------------+-------------+-------------+------------+-------------+-----------------------+-------------+-----------------+----------------------+----------------------+----------------------+------------------+-------------------+-----------------+--------------------+--------------+-------------+-------------+---------------+--------------------+------------------------------------------+------------------------------------------------+-----------------------------------+-----------------------+----------------------+-------------------------+---------------------------+---------------+-----------------------+----------------------+----------------------------+---------------------+-----------------------+-----------------+--------------------------+--------------------------+----

In [76]:
SQL = """ 
          Select * from FullMerge f left join
           Emissions e on f.name=e.reference_name and f.year= e.year left join Fundmapping fm
          where f.name=e.reference_name and f.year= e.year and f.name= fm.name
        """
results_full_merge_emission_fundmapping = spark.sql(SQL)
results_full_merge_emission_fundmapping.count()
results_full_merge_emission_fundmapping.show(10)

215894

+-----+----------+-------------+-----------------+-------------+----------------+-------------+------------------+-------------+----------------------+--------------------+----------------+-----------+----------------+-------------+-------------+------------+-------------+-----------------------+-------------+-----------------+----------------------+----------------------+----------------------+------------------+-------------------+-----------------+--------------------+--------------+-------------+------------+---------------+--------------------+------------------------------------------+------------------------------------------------+-----------------------------------+-----------------------+----------------------+-------------------------+---------------------------+-------------+-----------------------+----------------------+----------------------------+---------------------+-----------------------+-----------------+--------------------------+--------------------------+--------

In [ ]:
def create_spark_df(FullMergeDf, emissions_df, FundMapping_df, categorical_cols, numerical_cols):
   SQL = """ 
          Select * from FullMerge f, Emission e, Fundmapping fm
          where f.name =e.Name and f.year= e.year and f.name= fm.name
        """
    results = spark.sql(SQL)
    return results

In [46]:
def create_df(FullMergeDf, emissions_df, FundMapping_df, categorical_cols, numerical_cols):
    """Takes as input the FullMerge dataframe and outputs an new dataframe transformed"""
    partitionsforDask = 20
    # add year
    # FullMergeDf['year'] = pd.to_datetime(FullMergeDf['date_x'])
    # FullMergeDf['year'] = pd.DatetimeIndex(FullMergeDf['date_x']).year
    FullMergeDf[categorical_cols] = FullMergeDf[categorical_cols].astype('category')
    # FullMergeDf[numerical_cols] = FullMergeDf[numerical_cols].astype(float) #test not adding it but if we use it we need to remove value from it then add it back
    FullMergeDf['year'] = FullMergeDf['year'].astype(int)
    FullMergeDf['Name'] = FullMergeDf['Name'].astype('category')

    #change the variable names - easier to do here than dask
    emissions_df.rename(columns={'reference_name': 'Name', 'carbonOutput': 'value'}, inplace=True)
    emissions_df['year'] = emissions_df['year'].astype(int)
    emissions_df['Name'] = emissions_df['Name'].astype('category')

    #convet to dask
    # FullMergeDf =  dd.from_pandas(FullMergeDf, npartitions=partitionsforDask) #.reset_index() # cudf.DataFrame.from_pandas(FullMerge)  
    # emissions_df =  dd.from_pandas(emissions_df, npartitions=partitionsforDask) #.reset_index() #cudf.DataFrame.from_pandas(emissions)    

    #set catergorial values before merging so process is quick
    # FullMergeDf[categorical_cols] = FullMergeDf[categorical_cols].astype('category').categorize()
    # FullMergeDf = FullMergeDf.categorize()

    #set indexs
    FullMergeDf = FullMergeDf.set_index("index")  #.reset_index() # cudf.DataFrame.from_pandas(FullMerge)  
    emissions_df =  emissions_df.set_index("index")

    # map carbon to MergeTable #set it to value
    FullMergeDf = dd.merge(FullMergeDf, emissions_df, how='left', on=['Name','year'])

    #ADD NEW features
    FullMergeDf, numerical_cols = intialFeatureEng(FullMergeDf, numerical_cols)

    # this is for upwork CSV file format
    FundMapping_df['embedding'] = FundMapping_df['embedding'].apply(
        convert_array)  # convert mapping to numpy array
    FundMapping_df['AllFundsAVGembedding'] = FundMapping_df['AllFundsAVGembedding'].apply(
        convert_array)  # convert mapping to numpy array
    #convert fundmapping to dask
    FundMapping_df =  dd.from_pandas(FundMapping_df, npartitions=partitionsforDask).reset_index()  # cudf.DataFrame.from_pandas(FundMapping_df)   #new

    # Merging above dataframe (FullMerge merged with Emissions) with FundMapping Dataset
    Full_df = dd.merge(FullMergeDf, FundMapping_df, how='left', on=['Name'])


    return Full_df, numerical_cols

In [44]:
NumericColumns


['value',
 'totalAssets',
 'intangibleAssets',
 'earningAssets',
 'otherCurrentAssets',
 'totalLiab',
 'totalStockholderEquity',
 'deferredLongTermLiab',
 'otherCurrentLiab',
 'commonStock',
 'retainedEarnings',
 'otherLiab',
 'goodWill',
 'otherAssets',
 'cash',
 'totalCurrentLiabilities',
 'shortTermDebt',
 'shortLongTermDebt',
 'shortLongTermDebtTotal',
 'otherStockholderEquity',
 'propertyPlantEquipment',
 'totalCurrentAssets',
 'longTermInvestments',
 'netTangibleAssets',
 'shortTermInvestments',
 'netReceivables',
 'longTermDebt',
 'inventory',
 'accountsPayable',
 'totalPermanentEquity',
 'noncontrollingInterestInConsolidatedEntity',
 'temporaryEquityRedeemableNoncontrollingInterests',
 'accumulatedOtherComprehensiveIncome',
 'additionalPaidInCapital',
 'commonStockTotalEquity',
 'preferredStockTotalEquity',
 'retainedEarningsTotalEquity',
 'treasuryStock',
 'accumulatedAmortization',
 'nonCurrrentAssetsOther',
 'deferredLongTermAssetCharges',
 'nonCurrentAssetsTotal',
 'capital

In [47]:
df, NumericColumns = create_df(FullMergeDf = FullMerge, emissions_df=emissions,FundMapping_df = FundMapping, categorical_cols = catgoricalValues, numerical_cols = NumericColumns)


NameError: ignored

In [ ]:
def process_full_df(Full_df, categoricalValues, NumericColumns, vector):
    print("loading data...")
    orginalData = Full_df[categoricalValues + NumericColumns + vector]  # .copy()
    orginalData['periodDate'] = orginalData['year']
    #TEST
    orginalData = orginalData.categorize(columns=categoricalValues)

    # some tickers occur in data twice as upper/lower case.
    orginalData['ticker'] = orginalData['ticker'].str.lower()
    # remove dups #TODO drop duplicates but also look at vectors.
    # orginalData = orginalData[~orginalData.duplicated(
    #     subset=['ticker', 'periodDate'], keep='first')]

    # dups = orginalData[orginalData.duplicated(
    #     subset=['ticker', 'periodDate'], keep=False)]

    # set type for dataset
    orginalData[categoricalValues] = orginalData[categoricalValues].astype(
        'category')
    orginalData[NumericColumns] = orginalData[NumericColumns].astype('float64')

    # remember which ones were not null so we can call them back later when training after all data has been encoded
    labeled_inds = orginalData['value'].notnull()

    # For missing values, we are computing modes for categorical columns and means for numerical columns
    # modes = orginalData.select_dtypes(exclude=np.number).mode().iloc[0]
    # means = orginalData.mean()
    # both = modes.append(means)
    # new version is only filling for catgeroical and numerical values
    # orginalData[categoricalValues +
    #             NumericColumns] = orginalData[categoricalValues + NumericColumns].fillna(both)
    orginalData['periodDate'] = orginalData['periodDate'].fillna(
        orginalData['year'])  # for some reason year is missing for some items
    return orginalData, labeled_inds


In [ ]:
from utils_dask import process_full_df

categoricalValues = catgoricalValues
NumericColumns = NumericColumns
vector = vector

#change the column name - there's a bug right now that's duplicating these names #TODO investigate
df = df.rename(columns={'Exchange_x': 'Exchange', 'CountryName_x': 'CountryName'})

#set column types + do imputation
new_df, labeled_inds = process_full_df(Full_df = df, categoricalValues = categoricalValues , NumericColumns = NumericColumns, vector = vector)
##clear memory
# del emissions
# del FundMapping
print(f"Shape of new dataframe: {new_df.shape}")